#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

## Задачи для совместного разбора

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [1]:
import torch

In [2]:
class Neuron:
  def __init__(self, weights, bias):
    # <создать атрибуты объекта weights и bias>
    self.weights = weights
    self.bias = bias

  def forward(self, inputs):
    return inputs @ self.weights + self.bias


In [3]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

In [4]:
neuron = Neuron(weights, bias)
neuron.forward(inputs)

tensor(4.8400)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [5]:
class ReLU:
  def forward(self, inputs):
    # <реализовать логику ReLU>
    return torch.clip(inputs, min=0)

In [6]:
t = torch.normal(0, 1, size=(4,3))
t

tensor([[ 0.1288,  0.5472,  0.1669],
        [-0.6522, -0.2906,  0.4000],
        [-0.3819, -1.7617, -1.3270],
        [-1.3782, -1.0639, -0.8803]])

In [7]:
ReLU().forward(t)

tensor([[0.1288, 0.5472, 0.1669],
        [0.0000, 0.0000, 0.4000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [8]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return ((y_pred - y_true) ** 2).mean()

In [9]:
y_pred = torch.tensor([1.0, 3.0, 5.0])
y_true = torch.tensor([2.0, 3.0, 4.0])

In [10]:
MSELoss().forward(y_pred, y_true)

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1_1"></p>

1\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [11]:
import torch

In [12]:
class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):
        self.n_neurons = n_neurons
        self.n_features = n_features
        self.weights = torch.normal(0, 1, size=(n_neurons, n_features))
        self.bias = bias
        self.bias_in = torch.randint(1, 5, (n_neurons,))

    def forward(self, inputs):
        res = inputs @ self.weights.T
        if self.bias:
          res += self.bias_in
        return res

In [13]:
# inputs: batch_size(3) x 4
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [14]:
inputs.shape

torch.Size([3, 4])

In [15]:
Linear(3, 4, bias=True).forward(inputs)

tensor([[ 4.6258,  3.0188, -2.4332],
        [ 1.8446,  4.1553, -6.5128],
        [ 5.4839,  6.1562,  1.6785]])

<p class="task" id="1_2"></p>

1\.2 Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

In [16]:
l1 = Linear(inputs.shape[0], inputs.shape[1], bias=True).forward(inputs)
l2 = Linear(7, l1.shape[1], bias=True).forward(l1)
l2.shape

torch.Size([3, 7])

### Создание функций активации

<p class="task" id="2_1"></p>

2\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

In [17]:
class Softmax:
    def forward(self, inputs):
        return torch.exp(inputs) / torch.sum(torch.exp(inputs), axis=1, keepdim=True)

In [18]:
x = torch.normal(0, 1, size=(4,3))
Softmax().forward(x)

tensor([[0.2335, 0.2796, 0.4869],
        [0.4528, 0.3978, 0.1494],
        [0.0788, 0.0122, 0.9090],
        [0.2138, 0.3169, 0.4693]])

<p class="task" id="2_1"></p>

2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [19]:
class ELU:
    def __init__(self, alpha: float):
        self.alpha = alpha

    def forward(self, inputs):
        return torch.where(inputs >= 0, inputs, self.alpha * (torch.exp(inputs)))

In [20]:
inputs = torch.normal(0, 1, size=(4, 3))
inputs, ELU(0.3).forward(inputs)

(tensor([[ 0.5301, -0.1406, -1.0941],
         [-1.8102,  1.3156,  0.7732],
         [ 1.0691, -0.9695,  0.8420],
         [-0.5733, -1.5902, -0.2566]]),
 tensor([[0.5301, 0.2607, 0.1005],
         [0.0491, 1.3156, 0.7732],
         [1.0691, 0.1138, 0.8420],
         [0.1691, 0.0612, 0.2321]]))

### Создание функции потерь

<p class="task" id="3_1"></p>

3\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$
<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

In [29]:
class CrossEntropyLoss:
  def forward(self, y_pred, y_true):
    L = - torch.sum(y_true * torch.log(y_pred), axis=1)
    return 1 / len(y_pred) * sum(L)

In [30]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                        [2, 5, -1, 2],
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

In [31]:
l1 = Linear(inputs.shape[0], inputs.shape[1], bias=True).forward(inputs)

In [24]:
y_pred = Softmax().forward(l1)
y_pred

tensor([[6.0490e-01, 2.3780e-01, 1.5730e-01],
        [6.5366e-01, 2.2369e-04, 3.4612e-01],
        [1.7597e-05, 9.9801e-01, 1.9760e-03]])

In [25]:
CrossEntropyLoss().forward(y_pred, y)

tensor(3.9586)

<p class="task" id="3_2"></p>

3.2 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

In [26]:
class MSERegularized:
    def __init__(self, lambda_):
        self.lambda_ = lambda_

    def data_loss(self, y_pred, y_true):
        # <подсчет первого слагаемого из формулы>
        return ((y_pred - y_true) ** 2).mean()

    def reg_loss(self, weights):
        # <подсчет второго слагаемого из формулы>
        return self.lambda_ * sum(weights ** 2)

    def forward(self, y_pred, y_true, weights):
        return self.data_loss(y_pred, y_true) + self.reg_loss(weights)

In [27]:
y_pred = torch.tensor([-0.5, 1, 1.7])
y_true = torch.tensor([0, 0.6, 2.3])
weights = torch.normal(0, 5, (10, 1))

In [28]:
MSERegularized(2).forward(y_pred, y_true, weights)

tensor([613.4919])

## Обратная связь
- [x] Хочу получить обратную связь по решению